<a href="https://colab.research.google.com/github/kerryqliu/stats170/blob/irene/tf_idf_with_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sqlalchemy import create_engine

In [ ]:
# engine

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
query = "SELECT * from fullDocuments f join documentLabels d on f.docid = d.docid;"
fulldocs = pd.read_sql_query(query, con=engine)
fulldocs

,docid,fulltext,docid,caseid,label
0,1.0,Document Text: Example 1\nReferring Doctor: Un...,1,-1,1.0
1,2.0,Document Text: Example 2\nProgress Notes\nDate...,2,-1,1.0
2,3.0,Document Text: Example 3\nChief Complaint:\n1....,3,-1,4.0
3,4.0,Document Text: 14\nse\nLAIDA\n*\nPatient Infor...,4,11594,1.0
4,5.0,Document Text: The first set of documents behi...,5,11594,1.0
...,...,...,...,...,...
110,111.0,Document Text: 9:10\nFAX\n2002/041\n'\nPatient...,111,87832,1.0
111,112.0,Document Text: # 23/24\nfined VI\nvyy\nScan on...,112,88178,1.0
112,113.0,Document Text: 15:03 From\nPage: 1/24\nNUFACTO...,113,95139,1.0
113,114.0,Document Text: 03:58:42 p.m.\n1/2\nNU FACTOR\n...,114,95739,2.0


In [ ]:
corpus_dict = {}
corpus = []
tags = []
for i, row in fulldocs.iterrows():
  corpus.append(row['fulltext'])
  tags.append(row['label'])
  corpus_dict[i] = (row['fulltext'], row['label'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

### tf-idf

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(corpus, tags, test_size=0.2, random_state=10)
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train, labels_train)
features_train = selector.transform(features_train).toarray()
features_test = selector.transform(features_test).toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
model = LogisticRegression(C=100).fit(features_train, labels_train)
score = model.score(features_test, labels_test)
print(score)

0.4782608695652174


### Adjusting C, which determines the amount of regularization.
Lower values increase regularization

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
tfidf_search = GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)

In [ ]:
Cs = [1e-7, 1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]

for c in Cs:
  curr_model = LogisticRegression(C=c).fit(features_train, labels_train)
  curr_score = curr_model.score(features_test, labels_test)
  print(f'{c}: {curr_score}')

1e-07: 0.43478260869565216
1e-05: 0.43478260869565216
0.001: 0.43478260869565216
0.1: 0.43478260869565216
1.0: 0.43478260869565216
10.0: 0.4782608695652174
100.0: 0.4782608695652174


highest accuracy:
max_df = 0.5 or without this, C = 100, 0.4782608695652174

considering different parameters:
- penalty
- C
- class_weight
- solver: 'newton-cg', 'sag', 'saga', 'lbfgs'
- max_iter

### with bigrams

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(corpus, tags, test_size=0.2, random_state=10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (1,2))
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [ ]:
for c in Cs:
  curr_model = LogisticRegression(C=c).fit(features_train, labels_train)
  curr_score = curr_model.score(features_test, labels_test)
  print(f'{c}: {curr_score}')

1e-07: 0.43478260869565216
1e-05: 0.43478260869565216
0.001: 0.43478260869565216
0.1: 0.43478260869565216
1.0: 0.43478260869565216
10.0: 0.43478260869565216
100.0: 0.4782608695652174


### with penalty l1

In [ ]:
l1_model = LogisticRegression(penalty = 'l1', solver = 'saga').fit(features_train, labels_train)
l1_score = l1_model.score(features_test, labels_test)
print(l1_score)

NameError: ignored

In [ ]:
Cs = [1e-7, 1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]

for c in Cs:
  curr_model = LogisticRegression(penalty = 'l1', solver = 'saga', C=c).fit(features_train, labels_train)
  curr_score = curr_model.score(features_test, labels_test)
  print(f'{c}: {curr_score}')

1e-07: 0.43478260869565216
1e-05: 0.43478260869565216
0.001: 0.34782608695652173
0.1: 0.43478260869565216
1.0: 0.43478260869565216


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


10.0: 0.4782608695652174


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


100.0: 0.4782608695652174


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


100000.0: 0.4782608695652174


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


1000000.0: 0.4782608695652174
10000000.0: 0.4782608695652174


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


### stratified kfold split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold 
# features_train, features_test, labels_train, labels_test = train_test_split(corpus, tags, test_size=0.2, random_state=10)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
# features_train = vectorizer.fit_transform(features_train)
# features_test = vectorizer.transform(features_test)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
corpus_np = np.array(corpus)
tags_np = np.array(tags)

In [ ]:
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)
vectorizer = TfidfVectorizer(stop_words='english')
accuracy_scores = []
train_dict = defaultdict(list)
test_dict = defaultdict(list)
train_f1_dict = defaultdict(list)
test_f1_dict = defaultdict(list)
vectorizer = TfidfVectorizer(stop_words='english')
accuracy_scores = []
Cs = [1e-7, 1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5] #, 1e6, 1e7]

for c in Cs:
  print(f'-----{c}')

  for train_index, test_index in skf.split(corpus, tags):
    # split corpus into train and test
    features_train, features_test = corpus_np[train_index], corpus_np[test_index]
    # split tags into train and test
    labels_train, labels_test = tags_np[train_index], tags_np[test_index]
    # train tfidf vectorizer
    features_train = vectorizer.fit_transform(features_train)
    features_test = vectorizer.transform(features_test)

    current = []

    curr_model = LogisticRegression(penalty = 'l1', solver = 'saga', C=c).fit(features_train, labels_train)
    # curr_score = curr_model.score(features_test, labels_test)

    labels_pred = curr_model.predict(features_test)
    labels_train_pred = curr_model.predict(features_train)

    train_dict[c].append(accuracy_score(labels_train, labels_train_pred))
    test_dict[c].append(accuracy_score(labels_test, labels_pred))
    train_f1_dict[c].append(f1_score(labels_train, labels_train_pred, average='weighted'))
    test_f1_dict[c].append(f1_score(labels_test, labels_pred, average='weighted'))

    # print(f'{c}: {curr_score}')
    current.append(curr_score)

  accuracy_scores.append(current)

-----1e-07
-----1e-05
-----0.001
-----0.1
-----1.0
-----10.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

-----100.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

-----1000.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

-----10000.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

-----100000.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

In [ ]:
train_dict

defaultdict(list,
            {1e-07: [0.6105263157894737,
              0.1875,
              0.6041666666666666,
              0.6041666666666666,
              0.6145833333333334,
              0.6145833333333334],
             1e-05: [0.6105263157894737,
              0.6041666666666666,
              0.6041666666666666,
              0.6041666666666666,
              0.6145833333333334,
              0.6145833333333334],
             0.001: [0.6105263157894737,
              0.6041666666666666,
              0.6041666666666666,
              0.1875,
              0.6145833333333334,
              0.6145833333333334],
             0.1: [0.6105263157894737,
              0.6041666666666666,
              0.6041666666666666,
              0.6041666666666666,
              0.6145833333333334,
              0.6145833333333334],
             1.0: [0.6105263157894737,
              0.6041666666666666,
              0.6041666666666666,
              0.6041666666666666,
              0.614